In [ ]:
# imports
import os
import json
import glob
from pathlib import Path
import xml.etree.ElementTree as et
import regex
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
import time
import zipfile
import io

# helper functions and constants
from dataGeneration.extract_contributions import extract
from dataGeneration.clean_text import clean_name_headers
from dataGeneration.match_names import insert_politician_id_into_contributions_extended
import paths as PATHS

# **6 Merge and Structure Speech & Contribution Data**


Merges all speech and contribution data from the 19th and 20th electoral periods into unified datasets. It also ensures consistency in structure, adds helpful metadata and saves all outputs both as .pkl and .xlsx files for downstream processing or analysis.

**Speech Content (Redebeiträge)**:
- Loads pickled DataFrames containing speech data per term.
- Adds the electoral_term and document_url fields for each speech.
- Ensures correct data types and column structure.
- Merges the 19th and 20th terms into one speech_content DataFrame.
- Saves:
    - Combined dataset (speech_content_19_20.pkl)
    - Per-period datasets (speech_content_19.pkl, speech_content_20.pkl)
    - Excel copies for all three.

**Extended Contributions (Einwürfe, Zwischenrufe)**:
- Iterates over all .pkl contribution files for each term.
- Renames and reorders relevant columns.
- Adds a unique contribution id.
- Ensures unified data types.
- Saves:
    - Per-period contribution data as .pkl and .xlsx.
    - A merged dataset contributions_extended_19_20.pkl.


### **Input:**
```
rawData/
├── rawData19xml/*.xml
│   ├── 19001.xml
│   └── …
├── rawData20xml/*.xml
│   ├── 20001.xml
│   └── …
dataStage04/
├── speechContent/
│   ├── electoral_term_19/
│   │   └── speech_content.pkl
│   ├── electoral_term_20/
│   │   └── speech_content.pkl
dataStage06/
├── contributionsExtendedStage06/
│   ├── electoral_term_19/
│   │   ├── 19001.pkl
│   │   └── …
│   ├── electoral_term_20/
│   │   ├── 20001.pkl
│   │   └── …
```

### **Ouput:**
```
dataFinalStage/
├── contributionsExtendedFinalStage/
│   ├── contributions_extended_19_20.pkl
│   ├── contributions_extended_19.pkl
│   └── contributions_extended_20.pkl
├── speechContentFinalStage/
│   ├── speech_content_19_20.pkl
│   ├── speech_content_19.pkl
│   └── speech_content_20.pkl
dataExcel/
├── finalStage
│   ├── contributions_extended_19_20_finalStage.xlsx
│   ├── contributions_extended_19_finalStage.xlsx
│   ├── contributions_extended_20_finalStage.xlsx
│   ├── speech_content_19_20_finalStage.xlsx
│   ├── speech_content_19_finalStage.xlsx
│   └── speech_content_20_finalStage.xlsx
```


**Columns (speech_content_*.pkl):**
| Column            | Description                                          |
|-------------------|------------------------------------------------------|
| `id`              | Unique ID for each speech block                     |
| `electoral_term`  | Electoral period (e.g., 19, 20)                     |
| `session`         | Session number within the term                      |
| `first_name`      | First name of the speaker                           |
| `last_name`       | Last name of the speaker                            |
| `faction_id`      | Faction ID assigned to the speaker                  |
| `position_short`  | Speaker's role (e.g., MP, Minister)                 |
| `position_long`   | Full position title (e.g., "Bundesminister für...") |
| `politician_id`   | Unique politician ID (ui)                           |
| `speech_content`  | Full speech text                                    |
| `date`            | Session date (in Unix time format)                  |
| `document_url`    | Link to original PDF of session                     |

**Columns (contributions_extended_*.pkl):**
| Column          | Description                                   |
|------------------|-----------------------------------------------|
| `id`            | Unique ID for the contribution                |
| `type`          | Contribution type (e.g., "Einwurf", "Beifall") |
| `first_name`    | First name of the contributor                 |
| `last_name`     | Last name of the contributor                  |
| `faction_id`    | Faction ID of the contributor                 |
| `speech_id`     | Refers to the speech this contribution belongs to |
| `text_position` | Position within the speech text               |
| `politician_id` | ID of the contributor (if identifiable)       |
| `content`       | Extracted contribution text                   |

In [ ]:
# input directory
RAW_XML = {
    19: PATHS.RAW_XML_19,
    20: PATHS.RAW_XML_20,
}
SPEECH_CONTENT_INPUT = PATHS.SPEECH_CONTENT_STAGE04
SPEECH_CONTENT_INPUT_2 = PATHS.SPEECH_CONTENT_04_19
SPEECH_CONTENT_INPUT_3 = PATHS.SPEECH_CONTENT_04_20
CONTRIBUTIONS_EXTENDED_INPUT = PATHS.CONTRIBUTIONS_EXTENDED_STAGE06

# output directory
SPEECH_CONTENT_OUTPUT = PATHS.FINAL_SPEECH_CONTENT
CONTRIBUTIONS_EXTENDED_OUTPUT = PATHS.FINAL_CONTRIB_EXT
EXCEL_OUTPUT = PATHS.EXCEL_FINAL_STAGE

# ensure output directories exist
EXCEL_OUTPUT.mkdir(parents=True, exist_ok=True)
SPEECH_CONTENT_OUTPUT.mkdir(parents=True, exist_ok=True)
CONTRIBUTIONS_EXTENDED_OUTPUT.mkdir(parents=True, exist_ok=True)

# spoken content

# Placeholder for concating speeches DF of all sessions.
speech_content_19_20 = []

meta_data = {}

# Open every xml plenar file in every legislature period.
for term_number, folder_path in RAW_XML.items():
    # Skip e.g. the .DS_Store file.
    if not folder_path.is_dir():
        print(f"WARNUNG: {folder_path} existiert nicht.")
        continue

    #term_number = regex.search(r"(?<=electoral_term_)\d{2}", folder_path.stem)
    #if term_number is None:
    #    continue
    #term_number = int(term_number.group(0))

    for xml_plenar_file_path in sorted(folder_path.glob("*.xml")):
        tree = et.parse(xml_plenar_file_path)
        # Get the document number, the date of the session and the content.
        # meta_data["document_number"].append(tree.find("NR").text)
        # meta_data["date"].append(tree.find("DATUM").text)
        # document_number = tree.find("NR").text
        #date = time.mktime(
        #    datetime.strptime(
        #        tree.find("Datum").text, "%d.%m.%Y"
        #    ).timetuple()
        #)
        root = tree.getroot()
        datum_text = root.attrib.get("sitzung-datum")
        if not datum_text:
            print(f"Kein sitzung-datum Attribut in Datei: {xml_plenar_file_path}")
            continue
        date = time.mktime(datetime.strptime(datum_text, "%d.%m.%Y").timetuple())
        document_number = xml_plenar_file_path.stem
        document_number = int(document_number)
        meta_data[document_number] = date


speech_content_19 = pd.read_pickle(
    SPEECH_CONTENT_INPUT_2 / "speech_content.pkl"
)
speech_content_20 = pd.read_pickle(
    SPEECH_CONTENT_INPUT_3 / "speech_content.pkl"
)

speech_content_19 = speech_content_19.loc[
    :,
    [
        "id",
        "session",
        "first_name",
        "last_name",
        "faction_id",
        "position_short",
        "position_long",
        "politician_id",
        "speech_content",
        "date",
    ],
]
speech_content_20 = speech_content_20.loc[
    :,
    [
        "id",
        "session",
        "first_name",
        "last_name",
        "faction_id",
        "position_short",
        "position_long",
        "politician_id",
        "speech_content",
        "date",
    ],
]


speech_content_19.insert(1, "electoral_term", -1)
speech_content_20.insert(1, "electoral_term", -1)

speech_content_19["electoral_term"] = speech_content_19["session"].apply(
    lambda x: str(x)[:2]
)
speech_content_20["electoral_term"] = speech_content_20["session"].apply(
    lambda x: str(x)[:2]
)
speech_content_19["session"] = speech_content_19["session"].apply(lambda x: str(x)[-3:])
speech_content_20["session"] = speech_content_20["session"].apply(lambda x: str(x)[-3:])

speech_content_19["document_url"] = speech_content_19.apply(
    lambda row: "https://dip21.bundestag.de/dip21/btp/{0}/{0}{1}.pdf".format(
        row["electoral_term"], row["session"]
    ),
    axis=1,
)
speech_content_20["document_url"] = speech_content_20.apply(
    lambda row: "https://dip21.bundestag.de/dip21/btp/{0}/{0}{1}.pdf".format(
        row["electoral_term"], row["session"]
    ),
    axis=1,
)

speech_content_19["electoral_term"] = speech_content_19["electoral_term"].astype(
    "int32"
)
speech_content_20["electoral_term"] = speech_content_20["electoral_term"].astype(
    "int32"
)
speech_content_19["session"] = speech_content_19["session"].astype("int32")
speech_content_20["session"] = speech_content_20["session"].astype("int32")

speech_content = pd.concat([speech_content_19, speech_content_20])

# save data.
speech_content.to_pickle(SPEECH_CONTENT_OUTPUT / "speech_content_19_20.pkl")
speech_content_19.to_pickle(SPEECH_CONTENT_OUTPUT / "speech_content_19.pkl")
speech_content_20.to_pickle(SPEECH_CONTENT_OUTPUT / "speech_content_20.pkl")

speech_content.to_excel(EXCEL_OUTPUT / "speech_content_19_20_finalStage.xlsx", index=False)
speech_content_19.to_excel(EXCEL_OUTPUT / "speech_content_19_finalStage.xlsx", index=False)
speech_content_20.to_excel(EXCEL_OUTPUT / "speech_content_20_finalStage.xlsx", index=False)

# Placeholder für contribs nach Wahlperiode
contribs_by_term = {}

# Walk over all legislature periods.
for folder_path in sorted(CONTRIBUTIONS_EXTENDED_INPUT.iterdir()):
    if not folder_path.is_dir():
        continue

    term_number_match = regex.search(r"(?<=electoral_term_)\d{2}", folder_path.stem)
    if term_number_match is None:
        continue
    term_number = int(term_number_match.group(0))

    contribs = []
    for contrib_ext_file_path in sorted(folder_path.glob("*.pkl")):
        contribs.append(pd.read_pickle(contrib_ext_file_path))

    contrib_df = pd.concat(contribs, sort=False)

    contrib_df = contrib_df.loc[
        :,
        [
            "type",
            "first_name",
            "last_name",
            "faction_id",
            "id",
            "text_position",
            "politician_id",
            "content",
        ],
    ]

    contrib_df = contrib_df.rename(columns={"id": "speech_id"})

    contrib_df.insert(0, "id", list(range(len(contrib_df))))
    contrib_df["first_name"] = contrib_df["first_name"].apply(" ".join)

    contrib_df = contrib_df.astype(
        {
            "id": "int64",
            "type": "object",
            "first_name": "object",
            "last_name": "object",
            "faction_id": "int32",
            "speech_id": "int32",
            "text_position": "int32",
            "politician_id": "int32",
            "content": "object",
        }
    )

    # Save per term
    contrib_df.to_pickle(CONTRIBUTIONS_EXTENDED_OUTPUT / f"contributions_extended_{term_number}.pkl")
    contrib_df.to_excel(EXCEL_OUTPUT / f"contributions_extended_{term_number}_finalStage.xlsx", index=False)

    # Add to global list
    contribs_by_term[term_number] = contrib_df

# Combine all contributions
contributions_extended = pd.concat(contribs_by_term.values(), sort=False)

contributions_extended.to_pickle(CONTRIBUTIONS_EXTENDED_OUTPUT / "contributions_extended_19_20.pkl")
contributions_extended.to_excel(EXCEL_OUTPUT / "contributions_extended_19_20_finalStage.xlsx", index=False)

print("Done.")